In [ ]:
%%capture
!pip install transformers datasets
!git clone https://github.com/sunbirdai/leb.git
!pip install -r leb/requirements.txt
!pip install jiwer

In [ ]:
import yaml
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import pipeline
from datasets import load_metric
import leb.dataset
import csv

In [ ]:
yaml_config = '''
common_source: &common_source
  type: speech
  language: [lug,eng,ach,nyn]
  preprocessing:
    - set_sample_rate:
        rate: 16_000

common_target: &common_target
  type: text
  language: [lug,eng,ach,nyn]
  preprocessing:
    - lower_case
    - clean_and_remove_punctuation

test:
    huggingface_load:
        - path: Sunbird/salt
          name: multispeaker-lug
          split: test
        - path: Sunbird/salt
          name: multispeaker-eng
          split: test
        - path: Sunbird/salt
          name: multispeaker-ach
          split: test
        - path: Sunbird/salt
          name: multispeaker-nyn
          split: test

    source: *common_source
    target: *common_target

'''

In [ ]:
auth_token = "xxxxx"


In [ ]:
with open("config.yaml", "a") as f:
  print(yaml_config, file=f)

In [ ]:
class TranscriptionEvaluator:
    """
    Evaluates transcription quality over multiple languages using a SB MMS models,
    measuring Word Error Rate (WER) across a test set for each language.
    """
    def __init__(self, config_path):
        with open(config_path) as f:
            self.config = yaml.safe_load(f)
        self.test_ds = leb.dataset.create(self.config['test'])
        self.supported_languages = ['ach', 'lug', 'teo', 'nyn']
        self.wer_metric = load_metric("wer")
        self.auth_token = os.environ.get("HF_TOKEN")
        os.environ["HF_TOKEN"] = "xxxxx"
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def transcribe_audio_batch(self, audio_files, languages):
        """
        Transcribes a batch of audio files, filtering unsupported languages.
        """
        transcriptions, success_flags = [], []
        for language in set(languages):
            if language not in self.supported_languages:
                print(f"Skipping unsupported language: {language}")
                continue
            transcriptions, success_flags = self.transcribe_for_language(audio_files, languages, language)
        return transcriptions, success_flags

    def transcribe_for_language(self, audio_files, languages, language):
        """
        Handles transcription for a specific language.
        """
        try:
            pipe = self.initialize_pipeline(language)
            lang_indices = [i for i, lang in enumerate(languages) if lang == language]
            lang_audio_files = [audio_files[i] for i in lang_indices]
            outputs = pipe(lang_audio_files)
            return self.process_outputs(outputs, lang_indices), [True] * len(lang_indices)
        except Exception as e:
            print(f"Error processing language {language}: {e}")
            return [], []

    def initialize_pipeline(self, language):
        """
        Initializes the pipeline for a given language.
        """
        model_id = "Sunbird/sunbird-mms"
        pipe = pipeline(model=model_id, device=self.device, token=self.auth_token)
        pipe.tokenizer.set_target_lang(language)
        pipe.model.load_adapter(language)
        return pipe

    def process_outputs(self, outputs, lang_indices):
        """
        Processes the pipeline outputs into transcriptions.
        """
        transcriptions = [None] * len(lang_indices)
        for i, output in enumerate(outputs):
            transcriptions[lang_indices[i]] = output["text"]
        return transcriptions

    def calculate_batch_wer(self, predictions, references):
        """
        Calculates the Word Error Rate (WER) for a batch of predictions and references.
        """
        return self.wer_metric.compute(predictions=predictions, references=references)

    def evaluate(self):
        """
        Evaluates the WER across the test set for each language and prints the results.
        """
        batch_size = 8
        loader = DataLoader(self.test_ds, batch_size=batch_size, collate_fn=lambda x: x)
        total_wer, total_files = 0, 0
        wer_by_language = {}

        # Open a CSV file to write the transcriptions
        with open('transcriptions_comparison.csv', mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Language', 'Predicted Transcription', 'True Transcription'])

            for batch in loader:
                audio_files = [np.array(item['source']) for item in batch]
                languages = [item['source.language'] for item in batch]
                true_transcripts = [item['target'] for item in batch]

                predicted_transcripts, success_flags = self.transcribe_audio_batch(audio_files, languages)
                filtered_true_transcripts = [t for t, s in zip(true_transcripts, success_flags) if s]

                if predicted_transcripts and filtered_true_transcripts:
                    print(predicted_transcripts)
                    print(filtered_true_transcripts)
                    batch_wer = self.calculate_batch_wer(predicted_transcripts,filtered_true_transcripts)
                    total_wer += batch_wer * len(filtered_true_transcripts)
                    total_files += len(filtered_true_transcripts)
                    self.update_language_wer(languages, success_flags, batch_wer, wer_by_language)

                    # Write each transcription pair to the CSV, along with its language
                    for language, pred, true in zip(languages, predicted_transcripts, filtered_true_transcripts):
                      writer.writerow([language, pred, true])

        self.print_results(wer_by_language, total_wer, total_files)

    def update_language_wer(self, languages, success_flags, batch_wer, wer_by_language):
        """
        Updates the WER statistics for each language based on the batch results.
        """
        filtered_languages = [lang for lang, success in zip(languages, success_flags) if success]
        for language in filtered_languages:
            if language not in wer_by_language:
                wer_by_language[language] = []
            wer_by_language[language].append(batch_wer)

    def print_results(self, wer_by_language, total_wer, total_files):
        """
        Prints the final WER results by language and overall.
        """
        for language, wers in wer_by_language.items():
            avg_wer = sum(wers) / len(wers)
            print(f"{language}: {avg_wer:.4f}")
        overall_wer = total_wer / total_files if total_files > 0 else 0
        print(f"Overall WER across the test set: {overall_wer:.4f}")

In [ ]:
evaluator = TranscriptionEvaluator("config.yaml")
evaluator.evaluate()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Some weights of the model checkpoint at Sunbird/sunbird-mms were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

['ekikola kyakasooli kya kyenvu wabula langi yakyo etera okuba eya kitaka wansi', 'buli gi amabala ameru ku bikoola byakasooli galeetebwa biwuka', 'emikolo kitundu ku bulamu', 'emikolo kitundu ku bulamu', 'ekivulu kyabakazanyyirizi kyabadde kitya', 'ekivulu kya baakazannyirizi kyabadde kitya', 'kolera bulwadde obuleetebwa obukyafu', 'kolera bulwadde obuleetebwa obukyafu']
['ekikoola kya kasooli kya kyenvu wabula langi yaakyo etera okuba eya kitaka wansi', 'ebikoola bya kasooli biriiriddwa ebisaanyi', 'emikolo kitundu ku bulamu', 'emikolo kitundu ku bulamu', 'ekivvulu kya bakazanyiikirizi kyabadde kitya', 'ekivvulu kya bakazanyiikirizi kyabadde kitya', 'kolera bulwadde obuleetebwa bukyafu', 'kolera bulwadde obuleetebwa bukyafu']


Some weights of the model checkpoint at Sunbird/sunbird-mms were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Sunbird/sunbird-mms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

['embuto ezamangu zireetedde omuwendo gwabaana abawala abawanduka mu ssomero okweyongera', 'okusaba kwokweyimirirwa ku kakalu kujja kuwulirwa kkooti ku lwokutaano', 'abantu bangi tebaddizibwa ssente ze bakozesa mu kulabikako mu kkooti', 'buli omu yali mweraliikirivu ku ngeri yokubalirira', 'amakolero agalina abakozi abasukka mu kkumi galowooza kusalako bakozi', 'kimenya mateeka omukozi okukola nga taweebwa luwummula', 'abalimi balina okuwewulwa kwebyo bye basaasaanyizaako okusobola okwongeza ku magoba gaabwe', 'ebisolo bikosebwa nnyo endwadde']
['embuto ezamangu zireetedde omuwendo gwabaana abawala abawanduka mu ssomero okweyongera', 'okusaba kwokweyimirirwa ku kakalu kujja kuwulirwa kkooti ku lwokutaano', 'abantu bangi tebaddizibwa ssente ze bakozesa mu kulabikako mu kkooti', 'buli omu yali mweraliikirivu ku ngeri yokubalirira', 'amakolero agalina abakozi abasukka mu kkumi galowooza kusalako bakozi', 'kimenya mateeka omukozi okukola nga taweebwa luwummula', 'abalimi balina okuwewulwa 

Some weights of the model checkpoint at Sunbird/sunbird-mms were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Sunbird/sunbird-mms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

...

['orikwejunisa amakuru nobaasa kuhika ahabantu baingi', 'abebembezi bediini bashemereire kworoba nokugira engyesho nungi', 'nekirungi nkooreka amashemererwa', 'kubaho okukora ekintu ekitashemereire oine kushaba okusaasirwa', 'buri murwaire weena naikirizibwa kugira omwishanja ryomwe', 'nibintu ki ebiteirweho kubaasa kuziibira endwara ya korona kujanjaara', 'abarikuguza ebintu omu katare abayehandiikiise nibetombaitwa ahabwoburofa oburi omu katare', 'twatungayo omurwaire wena eriizooba']
['orikwejunisa amakuru noobaasa kuhika aha bantu baingi', 'abebembezi bediini bashemereire kworoba nokugira engyesho nungi', 'nikirungi okworeka amashemererwa', 'ku orikukora ekintu ekitashemeire oine kushaba okusaasirwa', 'buri murwaire weena naikirizibwa kugira omujanjaabi omwe', 'ni bintu ki ebitairweho kubaasa kuzibira endwara ya korona kujanjaara', 'abarikuguza ebintu omu katare abayehandiikise nibetomboitwa ahabwoburofa oburi omu katare', 'twatungayo omurwaire weena erizooba']


Some weights of the model checkpoint at Sunbird/sunbird-mms were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Sunbird/sunbird-mms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

['endwara ya korona ryo nejanjaara aho naaho', 'obushumankuzi nenki', 'purofesa akashwijuma orupapura rwe rwobwegyese', 'tekinorogye nereetaho emiringo emisya endiiju eobyobushuubuzi', 'torindiriire nokugomisiriza okuhisya owaratuteireho amatara gahanguota', 'abahingi baingi nibahinga ebyokuguza', 'omwebembezi aheereize tiimu yehanga ekichonco ahabwookukora gye munonga', 'abahagizi bomupiira nibakunda tiimu erikusinga']
['endwara ya korona eriyo neejanjaara ahonaaho', 'obushumankuzi nenki', 'purofeesa akashwijuma orupapura rwe rwobwegyese', 'tekinorogye neereetaho emiringo misya endiijo eyebyobushubuzi', 'turindiriire nokugumisiriza okuhisya obu barituteeraho amataara gahanguuto', 'abahingi baingi nibahinga ebyokuguza', 'omwebembezi ahareize tiimu yeihanga ekiconco ahabwokukora gye munonga', 'abahagizi bomupiira nibakunda tiimu erikusinga']


Some weights of the model checkpoint at Sunbird/sunbird-mms were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Sunbird/sunbird-mms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

['enzigye ku zirikutaahirira ebihingwa zishiisha byona ebirikuruga omu buhingi']
['enzigye kuzirikutaahirira ebihingwa nizishiisha byona ebirikuruga omubuhingi']
lug: 0.1510
ach: 0.3025
nyn: 0.3872
Overall WER across the test set: 0.2765


In [ ]:
# evaluator = TranscriptionEvaluator("config.yaml")
# evaluator.evaluate()

<ipython-input-10-852bf07fcbcb>:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in 

Skipping unsupported language: eng


Some weights of the model checkpoint at Sunbird/sunbird-mms were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Sunbird/sunbird-mms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

Skipping unsupported language: eng


Some weights of the model checkpoint at Sunbird/sunbird-mms were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Sunbird/sunbird-mms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

Error processing language ach: list assignment index out of range


Some weights of the model checkpoint at Sunbird/sunbird-mms were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Sunbird/sunbird-mms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

Error processing language nyn: list assignment index out of range


Some weights of the model checkpoint at Sunbird/sunbird-mms were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Sunbird/sunbird-mms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

lug: 0.1375
ach: 0.2974
nyn: 0.3918
Overall WER across the test set: 0.2530


In [ ]:
evaluator.print_results()

TypeError: TranscriptionEvaluator.print_results() missing 3 required positional arguments: 'wer_by_language', 'total_wer', and 'total_files'